In [1]:
import numpy as np 
from matplotlib import pyplot as plt

plt.rcdefaults()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", figsize=(11.2, 6.3))
plt.rc("font", size=12)
plt.rc("axes", edgecolor="white")
plt.rc("legend", frameon=True, framealpha=0.8, facecolor="white", edgecolor="white")

# 1 Numerical Linear Algebra I

## Basics

### Problem statement
For $A=(a_{ij})\in\mathbb{K}^{n\times n}$ and $\ b=(b_i)\in\mathbb{K}^n$ find or determine the existence of $x=(x_i)\in\mathbb{K}^n$ such that $Ax=b,$ where $\mathbb{K}=\mathbb{R}$ or $\mathbb{C}.$

### Matrix Types
Diagonalmatrix, Nullmatrix, Einheitsmatrix, Permutationsmatrix, monomiale Matrix, Rang-1-Matrix, tridiagonale Matrix / Bandmatrix, Dreiecksmatrix, Hessenbergmatrix, unitäre / orthogonale Matrix, Spiegelungsmatrix, hermitesche / symmetrische Matrix, positiv / negativ (semi-)definite Matrix, dünnbesetzte Matrix, H-Matrix, diagonaldominante Matrix, M-Matrix

### Norms
Spektralnorm, Zeilensummennorm, Spaltensummennorm, 

### Results from Linear Algebra
$Ax=b$ has exactly one solution if and only if A is regular, which is equivalent to $\det A \neq 0$ and to $\textrm{rank} A = n.$ In this case $x=A^{-1}b.$
Furthermore using Cramer's Rule $x$ can be determined by calculating $x_i = \dfrac{\det A^{(i)}}{\det A} = \dfrac{1}{\det A} \textrm{adj}(A)b.$ $A^{(i)}$ is given by replacing the $i$-th column of $A$ with $b$ and $\textrm{adj}(A)_{jk} = (-1)^{j+k} S_{kj}(A)$ is the adjoint matrix of $A,$ where $S_{kj}(A) \in\mathbb{K}^{(n-1)\times (n-1)}$ is given by deleting the $k$-th row and $j$-th column of $A.$ 

### Direct Methods

#### Gauß Algorithm
Solving a system with a lower or ipper triangular matrix is straightforward using forward or back substitution respectively and a solution exists if no diagonal element is zero.

In [2]:
def solveLower(L, b):
    n = len(b)    
    x = np.zeros(n)
    
    for i in range(n):
        if L[i,i] == 0:
            raise Exception("Matrix is singular.")
        else:
            x[i] = (b[i] - sum(L[i,:i] * x[:i])) / L[i,i]
    
    return x

In [3]:
def solveUpper(U, b):
    return solveLower(U[::-1,::-1], b[::-1])[::-1] #solves flipped to lower triangular system

For an $n\times n$ triangular matrix there are $k-1$ multiplications, $k-1$ additions and 1 division in the $k$-th step. Thus for $n$ steps there are $\frac{n(n-1)}{2}$ multiplications, $\frac{n(n-1)}{2}$ additions and $n$ divisions. Since these operations are roughly equivalent in terms of complexity on a modern computer there are $n^2$ operations and therefore the complexity is $O(n^2).$

The Gauß algorithm first decomposes a given matrix into an upper and lower triangular matrix.

In [30]:
def LUdecomp(A):
    A = np.array(A)
    n = A.shape[0]
    L = np.eye(n)
    U = A
    
    for k in range(n-1):
        for j in range(k+1, n):
            L[j,k] = U[j,k] / U[k,k]
            U[j,k:] = U[j,k:] - L[j,k] * U[k,k:]
    
    return L, U

There are $2(n-k+1)$ operations in the inner loop of the LU decomposition which results in a complexity of $O(n^3).$ This naive approach is not suitable for all regular matrices since it fails if there is a zero on the diagonal of the given matrix, e.g. $\big(\begin{smallmatrix} 0 & 1 \\ 1 & 0 \end{smallmatrix}\big).$ Because of floating point arithmetic errors a matrix with a diagonal element very close to zero, i.e. smaller than the machine precision $\textrm{eps},$ may not get properly decomposed by the LU decomposition:

In [31]:
A = [[10**(-20),1],[1,1]]
L, U = LUdecomp(A)
L@U

array([[1.e-20, 1.e+00],
       [1.e+00, 0.e+00]])

gauß gauß gauß

In [16]:
def GaußAlgorithm(A, b):
    L, U = LUdecomp(A)
    print(L)
    print(U)
    y = solveLower(L, b)
    return solveUpper(U, y)

Since the higher order complexity comes from the LU decomposition the Gauß algorithm also has complexity $O(n^3).$

In [32]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
b = [1,3,4,7]
x = GaußAlgorithm(A, b)
A@x - b

[[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [4. 3. 1. 0.]
 [3. 4. 1. 1.]]
[[2 1 1 0]
 [0 1 1 1]
 [0 0 2 2]
 [0 0 0 2]]


array([0., 0., 0., 0.])

#### Pivoting
By permuting rows of $A$ we can get the instability under control. Observe that in the $k$-th step of the Gauß algorithm multiples of the $k$-th row are subtracted from the rows $k+1,...,n$ of $A$ to get zeros in the $k$-th entry of that row. In this step the product of an entry of the $k$-th column and an element of the $k$-th row is subtracted and is then divided by the so called pivot element $A_{kk}.$ There is specific reason for using the $k$-th row and column in this step. Using row $i$ and column $j$ for $k < i,j \le n$ results in the pivot element $A_{ij}.$ Choosing such a pivot element which is not equal to zero is called pivoting.

First we will consider pivoting just the columns

In [47]:
def LUdecompColumnPivot(A):
    A = np.array(A)
    n = A.shape[0]
    L = np.eye(n)
    P = np.eye(n)
    U = A
    
    for k in range(n-1):
        i = k + np.abs(U[k:,k]).argmax() #chooses pivot column index
        print(i)
        print("U",U)
        U[k,k:], U[i,k:] = U[i,k:], U[k,k:].copy() #swaps rows
        print("U",U)
        print("L",L)
        L[k,:k], L[i,:k] = L[i,:k], L[k,:k].copy()
        print("L",L)
        print("P",P)
        P[k,:], P[i,:] = P[i,:], P[k,:].copy()
        print("P",P)
        
        for j in range(k+1, n):
            L[j,k] = U[j,k] / U[k,k]
            U[j,k:] = U[j,k:] - L[j,k] * U[k,k:]
    
    return L, U, P

In [48]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
L, U, P = LUdecompColumnPivot(A)
L@U - P@A

2
U [[2 1 1 0]
 [4 3 3 1]
 [8 7 9 5]
 [6 7 9 8]]
U [[8 7 9 5]
 [4 3 3 1]
 [2 1 1 0]
 [6 7 9 8]]
L [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
L [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
P [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
P [[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
3
U [[ 8  7  9  5]
 [ 0  0 -1 -1]
 [ 0  0 -1 -1]
 [ 0  1  2  4]]
U [[ 8  7  9  5]
 [ 0  1  2  4]
 [ 0  0 -1 -1]
 [ 0  0 -1 -1]]
L [[1.   0.   0.   0.  ]
 [0.5  1.   0.   0.  ]
 [0.25 0.   1.   0.  ]
 [0.75 0.   0.   1.  ]]
L [[1.   0.   0.   0.  ]
 [0.75 1.   0.   0.  ]
 [0.25 0.   1.   0.  ]
 [0.5  0.   0.   1.  ]]
P [[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
P [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
2
U [[ 8  7  9  5]
 [ 0  1  2  4]
 [ 0  0 -1 -1]
 [ 0  0 -1 -1]]
U [[ 8  7  9  5]
 [ 0  1  2  4]
 [ 0  0 -1 -1]
 [ 0  0 -1 -1]]
L [[1.   0.   0.   0.  ]
 [0.75 1.   0.   0.  ]
 [0.25 0.   1.   0.  ]
 [0.5  0.

array([[ 0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.75, -0.25, -0.25],
       [ 0.  ,  0.75,  0.25,  0.25],
       [ 0.  ,  0.5 ,  0.5 ,  0.5 ]])

complexity, alternatively permutation matrix as vector

In [14]:
def GaußAlgorithmColumnPivot(A, b):
    L, U, P = LUdecompColumnPivot(A)
    y = solveLower(L, P@b)
    return solveUpper(U, y)

In [46]:
A = [[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]
b = [1,3,4,7]
x = GaußAlgorithmColumnPivot(A, b)
A@x - b

2
3
2
[[1.   0.   0.   0.  ]
 [0.75 1.   0.   0.  ]
 [0.25 0.   1.   0.  ]
 [0.5  0.   1.   1.  ]]
[[ 8  7  9  5]
 [ 0  1  2  4]
 [ 0  0 -1 -1]
 [ 0  0  0  0]]
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


Exception: Matrix is singular.